#Download and Unzip Dataset
#Install Libraries

In [1]:
! wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
! mkdir news
! unzip 10k-german-news-articles.zip?raw=true -d news

--2020-03-03 18:14:32--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:14:32--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:14:32--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.12

In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.5MB/s 
     |████████████████████████████████| 552kB 44.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=c7dc29f61a333ad42b4f0d5d4e8235148ec17d0359fcb0bbbabad7e5d20bcdd4
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=1e0f1067e34ffe8159ad9f81abf62faa3f488d50d4d8982f225e63f7a1ae116b
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [3]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from sklearn.decomposition import LatentDirichletAllocation as LDA
import scipy
warnings.simplefilter("ignore", DeprecationWarning)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
for dirname, _, filenames in os.walk('/content/news'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/news/CrossValSplit.csv
/content/news/Newspaper_Staff.csv
/content/news/Annotations_consolidated.csv
/content/news/Articles.csv
/content/news/Posts.csv
/content/news/Annotations.csv
/content/news/Categories.csv


In [0]:
n=None
annotations_consol = pd.read_csv("/content/news/Annotations_consolidated.csv", nrows=n)
articles = pd.read_csv("/content/news/Articles.csv", nrows=n)
cat = pd.read_csv("/content/news/Categories.csv", nrows=n)
posts = pd.read_csv("/content/news/Posts.csv", nrows=n)
crossval = pd.read_csv("/content/news/CrossValSplit.csv", nrows=n)
staff = pd.read_csv("/content/news/Newspaper_Staff.csv", nrows=n)
annotations = pd.read_csv("/content/news/Annotations.csv", nrows=n)

#Merge Datasets

In [0]:
posts.shape, articles.shape

((1011773, 10), (12087, 5))

In [0]:
result = pd.merge(posts, articles, on='ID_Article', how='inner')

In [0]:
result.shape

(1011773, 14)

In [0]:
posts

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes
0,1,NaN,1,9089,2003-04-23 14:52:41.870,deleted,NaN,NaN,0,0
1,2,NaN,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,0,0
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,0,0
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,0,0
4,5,NaN,1,3343,2004-03-02 11:37:44.100,online,NaN,und sie als mitarbeiter sind natuerlich objektiv,0,0
...,...,...,...,...,...,...,...,...,...,...
1011768,1011769,1011764.0,12087,6355,2016-06-01 23:10:29.003,online,NaN,zwischen der beendigung eines arbeitsverhältni...,0,0
1011769,1011770,1011765.0,12087,6355,2016-06-01 23:11:14.790,online,NaN,du sagst es ja im namen selbst: wegen eindicku...,1,0
1011770,1011771,1011770.0,12087,27023,2016-06-02 08:16:56.690,online,NaN,Was genau haben Sie denn nicht verstanden? Ich...,0,1
1011771,1011772,1011769.0,12087,19159,2016-06-02 09:12:09.993,online,NaN,irgendwie widersprechen Sie sich in Ihrem Post...,1,0


In [0]:
result = pd.merge(result, crossval, on='ID_Post', how='left')

In [0]:
result = pd.merge(result, staff, on='ID_User', how='left')

In [0]:
result = pd.merge(result, annotations, on='ID_Post', how='left')

In [0]:
result = pd.merge(result, annotations_consol, on='ID_Post', how='left')

In [0]:
result.shape

(5089860, 21)

Merged the datasets however it did not make sense to me to use as it has articles repeated multiple times as the primary key of posts is ID_Post and ID_Articles is not unique.
articles cannot be merged with any other csv directly as it does not share a key.

#Preprocess data

1.   Removed html tags
2.   Removed escape characters
3.   Remove characters except alphabetical(including German alpha chars) and whitespace characters
4.   Tokenize using WordPunctTokenizer coz German
5.   Remove stopwords
6.   Convert to lowercase



In [0]:
def preprocess(doc):
    sentences = []
    doc = re.sub(r'<.*?>', '', doc)
    doc = re.sub(r'\n', '', doc)
    doc = re.sub(r'\t', '', doc)
    doc = re.sub(r'\r', '', doc)
    doc = re.sub(r'[^A-Za-z\säöüÄÖÜß]', ' ', doc)
    stop_words = set(stopwords.words('german'))
    word_tokens = WordPunctTokenizer().tokenize(doc)
    t = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    t = ' '.join(t)
    return t

In [0]:
articles['cleanedbody'] = articles['Body'].apply(preprocess)
articles['cleanedtitle'] = articles['Title'].apply(preprocess)

In [12]:
articles.head()

,ID_Article,Path,publishingDate,Title,Body,cleanedbody,cleanedtitle
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",abonnieren unserer newsletter lassen aktuelle ...,newsletter derstandard at
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",teil derstandard at dezember function popupwin...,teil derstandard at
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",smartphone app bietet nachrichten echtzeit mög...,android app derstandard at
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",sieht alltag linkshänder erledigen arbeiten re...,erfahrungen linkshänder gemacht
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",erzählen erlebnissen rund weltpolitischen umbr...,jahr erlebt


In [0]:
def print_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [0]:
number_topics = 5
number_words = 5
def return_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    t=[]
    for topic_idx, topic in enumerate(model.components_):
        t.append(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return t

#TFIDF and LDA

In [13]:
title_vectorizer = TfidfVectorizer()
title = title_vectorizer.fit_transform(articles['cleanedtitle'])
body_vectorizer = TfidfVectorizer()
body = body_vectorizer.fit_transform(articles['cleanedbody'])

title_lda = LDA(n_components=number_topics, n_jobs=-1)
title_lda.fit(title)
body_lda = LDA(n_components=number_topics, n_jobs=-1)
body_lda.fit(body)

print("Title Topics:")
print_topics(title_lda, title_vectorizer, number_words)
print("\n\n")
print("Body Topics:")
print_topics(body_lda, body_vectorizer, number_words)

Title Topics:
Topic #0:
mehr flüchtlinge apple forum us
Topic #1:
wien zwei eu tote mehr
Topic #2:
österreich flüchtlinge wien neue apple
Topic #3:
chef eu euro salzburg österreich
Topic #4:
wien wiener mehr eu wegen



Body Topics:
Topic #0:
clarkson dada gemeindeergebnisse valverde gravitationswellen
Topic #1:
övgw georgiou flatout kopera grohl
Topic #2:
prozent mehr euro sei wurde
Topic #3:
volltext agenturmeldungen basierenden artikels rechtlichen
Topic #4:
mensdorff shabaab pouilly mogadischu suu


In [0]:
title_topics = return_topics(title_lda, title_vectorizer, number_words)
body_topics = return_topics(body_lda, body_vectorizer, number_words)

In [15]:
print(title_topics, body_topics)

['mehr flüchtlinge apple forum us', 'wien zwei eu tote mehr', 'österreich flüchtlinge wien neue apple', 'chef eu euro salzburg österreich', 'wien wiener mehr eu wegen'] ['clarkson dada gemeindeergebnisse valverde gravitationswellen', 'övgw georgiou flatout kopera grohl', 'prozent mehr euro sei wurde', 'volltext agenturmeldungen basierenden artikels rechtlichen', 'mensdorff shabaab pouilly mogadischu suu']


#LDA Visualization on 5000 rows
Running on entire dataset would take too long

In [0]:
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis

title_vectorizer_vis = TfidfVectorizer()
title_vis = title_vectorizer_vis.fit_transform(articles['cleanedtitle'][:500])
body_vectorizer_vis = TfidfVectorizer()
body_vis = body_vectorizer_vis.fit_transform(articles['cleanedbody'][:500])

title_lda_vis = LDA(n_components=number_topics, n_jobs=-1)
title_lda_vis.fit(title_vis)
body_lda_vis = LDA(n_components=number_topics, n_jobs=-1)
body_lda_vis.fit(body_vis)

LDAvis_prepared = sklearn_lda.prepare(title_lda_vis, title_vis, title_vectorizer_vis)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_title_prepared_'+ str(number_topics) +'.html')
LDAvis_prepared = sklearn_lda.prepare(body_lda_vis, body_vis, body_vectorizer_vis)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_body_prepared_'+ str(number_topics) +'.html')

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [0]:
# pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')

#Mapping topics back to articles

In [0]:
t_topics = []
b_topics = []
for t, b in zip(title, body):
    t_topic = title_topics[np.argmax(title_lda.transform(t))]
    b_topic = body_topics[np.argmax(body_lda.transform(b))]
    t_topics.append(t_topic)
    b_topics.append(b_topic)

In [0]:
articles['title_topic'] = t_topics
articles['body_topic'] = b_topics

In [19]:
articles.head()

,ID_Article,Path,publishingDate,Title,Body,cleanedbody,cleanedtitle,title_topic,body_topic
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",abonnieren unserer newsletter lassen aktuelle ...,newsletter derstandard at,wien neue flüchtlinge drei mehr,prozent mehr euro sei wurde
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",teil derstandard at dezember function popupwin...,teil derstandard at,tv radiotipps eu wien österreich,prozent mehr euro sei wurde
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",smartphone app bietet nachrichten echtzeit mög...,android app derstandard at,wien neue flüchtlinge drei mehr,prozent mehr euro sei wurde
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",sieht alltag linkshänder erledigen arbeiten re...,erfahrungen linkshänder gemacht,wien neue flüchtlinge drei mehr,prozent mehr euro sei wurde
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",erzählen erlebnissen rund weltpolitischen umbr...,jahr erlebt,tv radiotipps eu wien österreich,prozent mehr euro sei wurde


#Calculating similarity using TFIDF and cosine similarity

In [0]:
tfidf_vectorizer12 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer12.fit(pd.concat([articles['cleanedtitle'], articles['title_topic']]))
vec121 = tfidf_vectorizer12.transform(articles['cleanedtitle'])
vec122 = tfidf_vectorizer12.transform(articles['title_topic'])

tfidf_vectorizer23 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer23.fit(pd.concat([articles['title_topic'], articles['body_topic']]))
vec232 = tfidf_vectorizer23.transform(articles['title_topic'])
vec233 = tfidf_vectorizer23.transform(articles['body_topic'])

tfidf_vectorizer13 = TfidfVectorizer(analyzer='word')
tfidf_vectorizer13.fit(pd.concat([articles['cleanedtitle'], articles['body_topic']]))
vec131 = tfidf_vectorizer13.transform(articles['cleanedtitle'])
vec133 = tfidf_vectorizer13.transform(articles['body_topic'])

In [0]:
sim12 = []
sim23 = []
sim13 = []
for i in range(len(articles)):
  sim12.append(scipy.spatial.distance.cosine(vec121[i].toarray(), vec122[i].toarray()))
  sim23.append(scipy.spatial.distance.cosine(vec232[i].toarray(), vec233[i].toarray()))
  sim13.append(scipy.spatial.distance.cosine(vec131[i].toarray(), vec133[i].toarray()))

In [0]:
articles['x'], articles['y'], articles['z'], articles['scores'] = sim12, sim23, sim13, (np.array(sim12) + np.array(sim12) + np.array(sim12))/3

In [23]:
articles['scores'].mean()

0.9915482405229078

#Final Score: 0.9915482405229078